In [2]:
# importing the pandas library as pd for simple and easy access
import pandas as pd

In [3]:
# Assigning the variable names to the files to be read
emp1= r"C:\Users\Narasimha\Desktop\Agent2_sample\scrapers\data\raw\employees_past_week.csv"
emp2= r"C:\Users\Narasimha\Desktop\Agent2_sample\scrapers\data\raw\employees_present_week.csv"

In [4]:
# Define the columns to be used and their data types
USECOLS =[0,1,2,3,4]
DTYPES={ 0: "string",
         1: "string",
         2: "string",
         3: "string",
         4: "string"
       }

In [5]:
df1= pd.read_csv(emp1, usecols=USECOLS, dtype=DTYPES, index_col = False)
df2= pd.read_csv(emp2, usecols=USECOLS, dtype=DTYPES, index_col=False)

In [6]:
df1.columns= ["email_id", "name","role", "company_name", "company_location"]
df2.columns= ["email_id", "name","role", "company_name", "company_location"]

In [7]:
# clean data by removing leading/trailing spaces and converting to lowercase
for df in [df1,df2]:
    df["email_id"]= df["email_id"].str.lower().str.strip()
    df["name"]= df["name"].str.lower().str.strip()
    df["role"]= df["role"].str.lower().str.strip()
    df["company_name"]= df["company_name"].str.lower().str.strip()
    df["company_location"]= df["company_location"].str.lower().str.strip()

In [8]:
#merge only needed columns to identify changes
merged=df1[["email_id", "role", "company_name", "company_location"]].merge(
    df2[["email_id", "role", "company_name"]],
    on="email_id",
    how="inner",
    suffixes=("_old", "_new")
)
merged.head()

,email_id,role_old,company_name_old,company_location,role_new,company_name_new
0,john.doe@acme.com,software engineer,acme corporation,san francisco,software engineer,acme corporation
1,jane.smith@globex.com,product manager,globex inc.,new york,product manager,globex inc.
2,robert.j@initech.com,data analyst,initech,boston,data analyst,initech
3,emily.c@umbrella.com,marketing director,umbrella corp,london,marketing director,umbrella corp
4,michael.b@hooli.com,cto,hooli,palo alto,cto,hooli


In [9]:
# Identify changes in company_name and role
company_changed = merged["company_name_old"] != merged["company_name_new"]
role_changed = merged["role_old"] != merged["role_new"]
role_changed.head()  # it will store boolean values indicating if role has changed

0    False
1    False
2    False
3    False
4    False
dtype: boolean

In [10]:
#creation of new dataset with changes only
changed_rows= merged[company_changed | role_changed].copy()
changed_rows.loc[role_changed,"status"]= "role changed"
changed_rows.loc[company_changed, "status"]= "company_changed"
changed_rows.tail()

,email_id,role_old,company_name_old,company_location,role_new,company_name_new,status
65,sienna.b@crescent.com,operations lead,crescent solutions,new york,operations lead,crescent global,company_changed
84,logan.k@serenity.com,healthcare analyst,serenity health,london,healthcare analyst,serenity global,company_changed
123,laura.d@avant.com,business intelligence analyst,avant bi,london,business intelligence analyst,avant data,company_changed
124,julian.h@maxwell.com,investment analyst,maxwell investments,toronto,investment strategist,maxwell investments,role changed
138,tristan.h@datacore.com,data infrastructure engineer,datacore,boston,data solutions engineer,datacore,role changed


In [11]:
changed_rows["Company (Location)"] = (
    changed_rows["company_name_old"] + "(" + changed_rows["company_location"] + ")" ).copy()


In [12]:
new_dataset= changed_rows[["email_id", "Company (Location)", "role_old", "status"]].copy()

In [13]:
new_dataset

,email_id,Company (Location),role_old,status
16,chris.p@apex.com,apex solutions(dublin),it support specialist,company_changed
19,sophia.t@everest.com,everest consulting(zurich),business analyst,company_changed
21,victoria.n@lumina.com,lumina designs(new york),product designer,company_changed
22,mark.d@fusion.com,fusion robotics(tokyo),robotics engineer,company_changed
23,paula.s@quantum.com,quantum ai(singapore),machine learning engineer,company_changed
26,brian.c@altair.com,altair security(chicago),cybersecurity analyst,company_changed
30,rachel.p@cypher.com,cypher ai(london),ai ethics specialist,company_changed
47,hailey.d@monarch.com,monarch industries(london),product analyst,company_changed
50,colin.b@astro.com,astro cloud(chicago),cloud security specialist,company_changed
55,ariana.l@polar.com,polar studios(toronto),design lead,company_changed


In [16]:
new_dataset.to_csv("Analysis_output.csv", index=False)